In [ ]:
from nba_api.stats.static import players
from nba_api.stats.static import teams
import numpy as np
import pandas as pd
import matplotlib as ml
import matplotlib.pyplot as plt
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import playergamelogs
from nba_api.stats.endpoints import playercareerstats
years="2021-22"
seasonType='Regular Season'
allTeamId=pd.DataFrame(teams.get_teams())['id']



# Four Factors

In [ ]:
from nba_api.stats.endpoints import boxscorefourfactorsv2
from nba_api.stats.endpoints import boxscoresummaryv2
df=boxscorefourfactorsv2.BoxScoreFourFactorsV2(game_id='0022100406' ).get_data_frames()[1]
# eliminateList=[  x for x in df
#     if  x!="GAME_ID" and  x!="TEAM_ID" and  x!="TEAM_ABBREVIATION" and x!="TEAM_CITY" and x!="MIN" ]
# df.loc[:,eliminateList]
idHomeTeam=boxscoresummaryv2.BoxScoreSummaryV2(game_id='0022100001').get_data_frames()[7]

# EFG_PCT=[(All Field Goals Made) + 0.5*(3P Field Goals Made)]/(All Field Goal Attempts) (effective)
# OREB_PCT=(Offensive Rebounds)/[(Offensive Rebounds)+(Opponent’s Defensive Rebounds)]
#FTA_RATE= (Total Free Throws Attempted) / (Total Field Goal Attempted)
# TM_TOV_PCT=(Turnovers)*100)/ [(Field Goal Attempts)+(Free Throw Attempts*0.44)+(Assists)+(Turnovers)]
#OPP_OREB_PCT=DEFENSIVE REBOUNDING PERECENTAGE%
idHomeTeam


In [ ]:
count=0
tmp=[]
nameFile="\\MatchId"+years+".txt"
for tmId in allTeamId:
    df = leaguegamefinder.LeagueGameFinder(team_id_nullable=tmId,season_nullable=years,season_type_nullable=seasonType).get_data_frames()[0]
    
    if(count==0):
        tmp=df["GAME_ID"].tolist()
        count=1
    else:
        for x in df["GAME_ID"].tolist():
            if x not in tmp:
                tmp.append(x)
            

with open(r"D:\UniSergio\BasketScripts\WaB\BoxScoresFile"+nameFile, 'w') as f:
            for line in tmp:
                tmpS=str(line)
                f.write(tmpS)
                f.write('\n')







In [ ]:
MatchId=[]
nameFile="/MatchId"+years+".txt"

with open(r"/home/sergio/UniSergio/BasketScript/WaB/outcomebasketmatch/BoxScoresFile"+nameFile,'r') as f:
        fr=f.readlines()  
        for line in fr:
            line=line.replace('\n', '')
           
            # We take the first 650 match 
            if(int(line[-3:]) <=650 and int(line[-4])==0):
                MatchId.append(line)

chunk_size = 50
chunked_list = [MatchId[i:i+chunk_size] for i in range(0, len(MatchId), chunk_size)]



# TRADIONAL SPLIT

In [ ]:
from nba_api.stats.endpoints import boxscoretraditionalv2 
from nba_api.stats.endpoints import boxscoresummaryv2
import winsound

tmpHomeTeam=[]
count=0
BoxScores=[]
nameFile="\\BoxScore"+years+"traditional"+".txt"
for i in range(len(chunked_list)):
    tmpBox=[]
    for x in chunked_list[i]:    

        df=boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=x).get_data_frames()[1]
        idHomeTeam=boxscoresummaryv2.BoxScoreSummaryV2(game_id=x)
        
        


        df["HOME"]=0

        label=[2]
        if(df.at[0,'PTS'] > df.at[1,'PTS']):
            label=[1]

        if(df.at[0,"TEAM_ID"]==idHomeTeam):
            df.at[0,'HOME']=1
        else:
            df.at[1,'HOME']=1
        
        
        eliminateList=[  x for x in df
            if  x!="GAME_ID"  and  x!="TEAM_ABBREVIATION" and x!="TEAM_CITY"   and x!='PLUS_MINUS' and x!="TEAM_NAME" and x!='MIN' and # x!="TEAM_ID" and 
                x!='FGM'  and  x!='FGA' and x!='FG3A' and x!='FG3M'and x!='FTM' and x!='FTA' and x!='REB'
            ]
            
        df=df.loc[:,eliminateList]
       
        tmp=list(df.loc[0])+list(df.loc[1])+label 
        
        tmpBox.append(tmp)
   
    with open(r"D:\UniSergio\BasketScripts\WaB\BoxScoresFile"+nameFile, 'a') as f:
        for line in tmpBox:
            tmpS=str(line)
            tmpS.replace("[","")
            tmpS.replace("]","")
            f.write(tmpS)
            f.write('\n')
    print("Finito il "+str(i)+"-esimo chunk ")


winsound.Beep(440, 2500)

# ADVANCE SPLIT

In [ ]:
from nba_api.stats.endpoints import boxscoreadvancedv2 
from nba_api.stats.endpoints import boxscoresummaryv2
import winsound

tmpHomeTeam=[]
count=0
BoxScores=[]
nameFile="\\BoxScore"+years+"advance"+".txt"
superCount=0
for i in range(len(chunked_list)):
    
    tmpBox=[]
    
    for x in chunked_list[i]:    
 
        df=boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=x).get_data_frames()[1]
        idHomeTeam=boxscoresummaryv2.BoxScoreSummaryV2(game_id=x)
  
        


        df["HOME"]=0

        label=[2]
        if(df.at[0,'NET_RATING'] > df.at[1,'NET_RATING']):
            label=[1]

        if(df.at[0,"TEAM_ID"]==idHomeTeam):
            df.at[0,'HOME']=1
        else:
            df.at[1,'HOME']=1
        eliminateList=[  x for x in df
            if  x!="GAME_ID"  and  x!="TEAM_ABBREVIATION"   and x!="TEAM_NAME" and x!='MIN' and # x!="TEAM_ID" and 
                x!="TEAM_CITY" and x.startswith("E_")==False
            ]
        df=df.loc[:,eliminateList]
        
        
        tmp=list(df.loc[0])+list(df.loc[1])+label 
        
        
        tmpBox.append(tmp)
    
    with open(r"D:\UniSergio\BasketScripts\WaB\BoxScoresFile"+nameFile, 'a') as f:
        for line in tmpBox:
            tmpS=str(line)
            tmpS.replace("[","")
            tmpS.replace("]","")

            f.write(tmpS)
            f.write('\n')
    print("Finito il "+str(i)+"-esimo chunk ")


winsound.Beep(440, 2500)

# Four Factors

In [ ]:
from turtle import home
from nba_api.stats.endpoints import boxscorefourfactorsv2
from nba_api.stats.endpoints import boxscoresummaryv2

tmpHomeTeam=[]
count=0
BoxScores=[]
nameFile="/BoxScore"+years+"fourfactors"+".txt"
superCount=0
for i in range(12,len(chunked_list)):
    
    tmpBox=[]
    
    for x in chunked_list[i]:    
 
        df=boxscorefourfactorsv2.BoxScoreFourFactorsV2(game_id=x).get_data_frames()[1]
        idHomeTeam=boxscoresummaryv2.BoxScoreSummaryV2(game_id=x).get_data_frames()[5]

        

    
        label=[1]

        if (idHomeTeam.at[1,"TEAM_WINS_LOSSES"][0]=='1'):
            label=[2]

            
        eliminateList=[  x for x in df
            if  x!="GAME_ID"  and  x!="TEAM_ABBREVIATION"   and x!="TEAM_NAME" and x!='MIN' and # x!="TEAM_ID" and 
                x!="TEAM_CITY" 
            ]
        df=df.loc[:,eliminateList]
        
    # [5] home
    # [6] Id_O        

        tmp=list(df.loc[0])+list(df.loc[1])+label 
        tmp1=list(df.loc[0])
        tmp1.insert(5,0.0)
        tmp1.insert(6,df.iloc[1]["TEAM_ID"])
        tmp1.insert(11,1.0)
        tmp1=tmp1+label
        print(tmp,idHomeTeam.at[1,"TEAM_WINS_LOSSES"])
        print(tmp1)
        tmpBox.append(tmp)
        break
    break
    # with open(r"/home/sergio/UniSergio/BasketScript/WaB/outcomebasketmatch/BoxScoresFile"+nameFile, 'a') as f:
    #     for line in tmpBox:
    #         tmpS=str(line)
    #         tmpS.replace("[","")
    #         tmpS.replace("]","")

    #         f.write(tmpS)
    #         f.write('\n')
    # print("Finito il "+str(i)+"-esimo chunk ")



# Reminder Advance Stats

In [ ]:

#TmPoss=0.5 * (FGA + 0.475 * FTA - ORB + TOV)
#Counter of team possession
#OppPoss=same of top, but for opposite team

#TS_PCT=Pts / (2 * (FGA + 0.475 * FTA))

#OFF_RATING= 100 / (TmPoss + OppPoss) * Pts

#DEF_RATING= 100 / (TmPoss + OppPoss) * OppPts
#NET_RATING=OFF_RATING -DEF_RATING
#Adjusts a team's scoring offense and defense for their pace of play

# df[df.columns.difference(["GAME_ID","TEAM_ID","TEAM_ABBREVIATION","TEAM_CITY","MIN"])]